In [1]:
import os
import numpy as np
import matplotlib
import tensorflow as tf
import cv2
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 1. 解决绘图后端问题 (保留你之前的 TkAgg 修复)
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.use('TkAgg', force=True)
import matplotlib.pyplot as plt

# 2. 导入 Waymo 数据定义
try:
    from waymo_open_dataset import dataset_pb2 as open_dataset
    # 尝试导入 E2E 数据定义
    from waymo_open_dataset.protos import end_to_end_driving_data_pb2 as wod_e2ed_pb2
    # ⚠️ 关键：导入官方 Demo 使用的相机投影算子
    from waymo_open_dataset.wdl_limited.camera.ops import py_camera_model_ops
    print(f"✅ 环境加载成功！")
    print(f"   - TensorFlow: {tf.__version__}")
    print(f"   - Camera Ops: 已加载")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    print("提示: 请确保安装了 waymo-open-dataset-tf-2-12-0 (或兼容版本)")

np.set_printoptions(precision=3, suppress=True)

2026-01-06 14:02:12.202471: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-06 14:02:12.203816: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-06 14:02:12.233262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-06 14:02:12.234503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-06 14:02:12.982318: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:3

✅ 环境加载成功！
   - TensorFlow: 2.13.0
   - Camera Ops: 已加载


In [2]:
import os
import tensorflow as tf
from google.protobuf.message import DecodeError

# 修改为你的实际文件路径
FILENAME = r'D:\Datasets\WOD_E2E_Camera_v1\val\val_202504211843.tfrecord-00008-of-00093'

# ---- 1) 路径兼容：Windows -> WSL（可选）
if (not os.path.exists(FILENAME)) and ('mnt' not in FILENAME) and (':' in FILENAME):
    FILENAME = FILENAME.replace('D:', '/mnt/d').replace('\\', '/')

print(f"📂 目标文件: {FILENAME}")

# ---- 2) 更稳的 exists 判断
if not tf.io.gfile.exists(FILENAME):
    raise FileNotFoundError(f"❌ 找不到文件: {FILENAME}")

# ---- 3) 自动尝试压缩类型（先按扩展名猜；不行再 fallback）
def make_tfrecord_dataset(path: str):
    # 先按后缀猜
    guess = "GZIP" if path.endswith(".gz") else ""
    tried = []
    for comp in [guess, "GZIP", ""]:
        if comp in tried:
            continue
        tried.append(comp)
        try:
            ds = tf.data.TFRecordDataset(path, compression_type=comp)
            # 触发读取一条，验证压缩类型对不对
            _ = next(iter(ds.take(1))).numpy()
            return ds, comp
        except Exception as e:
            last_err = e
    raise RuntimeError(f"❌ TFRecord 读取失败（已尝试压缩 {tried}），最后错误：{repr(last_err)}")

dataset, used_comp = make_tfrecord_dataset(FILENAME)
print(f"✅ TFRecordDataset 已就绪，compression_type='{used_comp}'")

# ---- 4) 读几条 raw bytes 做 sanity check
raw_list = [x.numpy() for x in dataset.take(3)]
print("📦 前 3 条 record bytes 长度:", [len(b) for b in raw_list])

# ---- 5) 尝试解析为 WOD-E2E proto（你已导入 wod_e2ed_pb2）
#     这里不假设具体 message 名字，自动枚举 pb2 里的 message class 逐个尝试 ParseFromString
def parse_as_any_e2e_message(raw_bytes, pb2_module):
    candidates = []
    for name, obj in vars(pb2_module).items():
        if isinstance(obj, type) and hasattr(obj, "DESCRIPTOR"):
            candidates.append((name, obj))

    # 优先尝试名字像 segment/frame/data 的
    priority = ["EndToEnd", "E2E", "Driving", "Segment", "Frame", "Data"]
    candidates.sort(key=lambda x: sum(p in x[0] for p in priority), reverse=True)

    for name, cls in candidates:
        msg = cls()
        try:
            msg.ParseFromString(raw_bytes)
            if len(msg.ListFields()) > 0:
                return name, msg
        except DecodeError:
            pass
    return None, None

# 你上面已经导入了：from waymo_open_dataset.protos import end_to_end_driving_data_pb2 as wod_e2ed_pb2
msg_name, msg = parse_as_any_e2e_message(raw_list[0], wod_e2ed_pb2)

if msg is None:
    print("⚠️ 读到了 bytes，但没法解析成 end_to_end_driving_data_pb2 里的任何 message。")
    print("   - 可能原因：压缩类型仍不对；或该 tfrecord 不是 E2E proto；或版本不匹配。")
else:
    print(f"✅ 解析成功: wod_e2ed_pb2.{msg_name}")
    print("🔎 顶层字段预览:", [fd.name for fd, _ in msg.ListFields()][:40])


📂 目标文件: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093
✅ TFRecordDataset 已就绪，compression_type=''
📦 前 3 条 record bytes 长度: [2275686, 1571030, 2162981]
✅ 解析成功: wod_e2ed_pb2.E2EDFrame
🔎 顶层字段预览: ['frame', 'future_states', 'past_states', 'intent', 'preference_trajectories']


2026-01-06 14:02:18.745337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-01-06 14:02:18.745599: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Block 1：基础导入 + 解析一条 E2EDFrame（最小起步）

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.protobuf.descriptor import FieldDescriptor

def parse_e2ed_frame_from_dataset(dataset):
    """从 TFRecordDataset 取第一条 record 并解析为 E2EDFrame"""
    raw = next(iter(dataset.take(1))).numpy()
    msg = wod_e2ed_pb2.E2EDFrame()
    msg.ParseFromString(raw)
    return msg

e2e = parse_e2ed_frame_from_dataset(dataset)
print("✅ Parsed E2EDFrame.")
print("Top-level set fields:", [fd.name for fd,_ in e2e.ListFields()])


✅ Parsed E2EDFrame.
Top-level set fields: ['frame', 'future_states', 'past_states', 'intent', 'preference_trajectories']


### Block 2：通用 Proto 检查函数（后面所有模块都复用）

In [4]:
def pb_set_fields(msg, title="", max_fields=80):
    """只打印当前 message 里被 set 的字段（ListFields）"""
    if title:
        print(f"\n=== {title} (set fields) ===")
    for fd, val in msg.ListFields()[:max_fields]:
        if fd.label == FieldDescriptor.LABEL_REPEATED:
            print(f"- {fd.name}: repeated len={len(val)}")
        else:
            if hasattr(val, "ListFields"):
                print(f"- {fd.name}: message<{fd.message_type.name}> subfields={len(val.ListFields())}")
            else:
                print(f"- {fd.name}: {val}")

def pb_all_fields(msg, title=""):
    """打印 proto 定义中的全部字段（不管 set 没 set）"""
    if title:
        print(f"\n=== {title} (all defined fields) ===")
    for f in msg.DESCRIPTOR.fields:
        rep = "repeated" if f.label == f.LABEL_REPEATED else "scalar"
        print(f"- {f.name}: {rep}, type={f.type}")

def pb_find_defined_fields(msg, keywords):
    """在“字段定义”里按关键词找字段名（不是 ListFields）"""
    ks = [k.lower() for k in keywords]
    hits = []
    for f in msg.DESCRIPTOR.fields:
        name = f.name.lower()
        if any(k in name for k in ks):
            hits.append(f.name)
    return hits

def enum_name(enum_wrapper, value):
    """稳定的 enum number -> name（避免 .Name() 版本差异）"""
    try:
        value_int = int(value)
    except Exception:
        value_int = value
    try:
        return enum_wrapper.DESCRIPTOR.values_by_number[value_int].name
    except Exception:
        return str(value)

def cam_name_from_id(cam_id):
    """CameraName enum -> string"""
    return enum_name(open_dataset.CameraName, cam_id) if hasattr(open_dataset, "CameraName") else str(cam_id)


### Block 3：frame 模块总览（frame 专用）

In [5]:
def inspect_frame_overview(frame):
    print("\n========== FRAME OVERVIEW ==========")
    pb_set_fields(frame, "frame", max_fields=60)

    # keyword search in defined fields
    print("\n[Defined-field search]")
    groups = [
        ("images",),
        ("context", "calibr"),
        ("pose",),
        ("lidar",),
        ("labels",),
        ("map",),
    ]
    for kws in groups:
        hits = pb_find_defined_fields(frame, kws)
        print(f"{kws} -> {hits}")

    # common quick stats
    if hasattr(frame, "images"):
        print("\nnum images:", len(frame.images))
    if hasattr(frame, "context") and hasattr(frame.context, "camera_calibrations"):
        print("num camera_calibrations:", len(frame.context.camera_calibrations))

frame = e2e.frame
inspect_frame_overview(frame)



========== FRAME OVERVIEW ==========

=== frame (set fields) ===
- context: message<Context> subfields=2
- timestamp_micros: 0
- images: repeated len=8

[Defined-field search]
('images',) -> ['images']
('context', 'calibr') -> ['context']
('pose',) -> ['pose', 'map_pose_offset']
('lidar',) -> ['projected_lidar_labels']
('labels',) -> ['laser_labels', 'projected_lidar_labels', 'camera_labels']
('map',) -> ['map_features', 'map_pose_offset']

num images: 8
num camera_calibrations: 8


### Block 4：frame 的 8 路相机图像提取 + 可视化（frame 专用）

In [6]:
def get_frame_images(frame):
    """
    返回：
      images: list[np.ndarray]  HWC uint8
      labels: list[str]         相机名
      meta:   list[dict]        cam_id/bytes_len/shape
    """
    images, labels, meta = [], [], []
    for img in frame.images:
        cam_id = img.name
        name = cam_name_from_id(cam_id)
        b = img.image
        arr = tf.io.decode_jpeg(b).numpy()
        images.append(arr)
        labels.append(name)
        meta.append({"cam_id": int(cam_id), "cam_name": name, "bytes_len": len(b), "shape": arr.shape})
    return images, labels, meta

def print_images_meta(meta):
    print("\n========== FRAME IMAGES META ==========")
    for m in meta:
        print(f"- {m['cam_name']} (id={m['cam_id']}): bytes={m['bytes_len']}, shape={m['shape']}")

def show_image_grid(images, labels, cols=4, figsize=(14, 8)):
    n = len(images)
    rows = int(np.ceil(n / cols))
    plt.figure(figsize=figsize)
    for i in range(n):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i])
        plt.title(labels[i], fontsize=10)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

images, labels, meta = get_frame_images(frame)
print_images_meta(meta)
show_image_grid(images, labels, cols=4)



========== FRAME IMAGES META ==========
- 2 (id=2): bytes=329031, shape=(1079, 972, 3)
- 1 (id=1): bytes=206775, shape=(1079, 972, 3)
- 3 (id=3): bytes=363232, shape=(1079, 972, 3)
- 4 (id=4): bytes=282652, shape=(1079, 972, 3)
- 5 (id=5): bytes=426742, shape=(1079, 972, 3)
- 8 (id=8): bytes=255339, shape=(587, 972, 3)
- 7 (id=7): bytes=202958, shape=(551, 972, 3)
- 6 (id=6): bytes=204414, shape=(587, 972, 3)


### Block 5：frame 的相机标定提取（intrinsics/extrinsics）（frame 专用）

In [7]:
def get_camera_calibration_map(frame):
    """返回 dict: cam_id -> CameraCalibration"""
    calib_map = {}
    if hasattr(frame, "context") and hasattr(frame.context, "camera_calibrations"):
        for c in frame.context.camera_calibrations:
            calib_map[c.name] = c
    return calib_map

def calib_summary(calib):
    out = {}
    if hasattr(calib, "width"): out["width"] = calib.width
    if hasattr(calib, "height"): out["height"] = calib.height
    if hasattr(calib, "intrinsic"):
        intr = list(calib.intrinsic)
        out["fx_fy_cx_cy"] = intr[:4] if len(intr) >= 4 else intr
    if hasattr(calib, "extrinsic") and hasattr(calib.extrinsic, "transform"):
        ex = list(calib.extrinsic.transform)
        if len(ex) == 16:
            out["extrinsic_4x4"] = np.array(ex, dtype=np.float32).reshape(4,4)
        else:
            out["extrinsic_flat"] = ex
    return out

def print_camera_calibrations(calib_map, max_print=8):
    print("\n========== CAMERA CALIBRATIONS ==========")
    for cam_id, calib in list(calib_map.items())[:max_print]:
        name = cam_name_from_id(cam_id)
        d = calib_summary(calib)
        print(f"\n[{name}] size=({d.get('width')}x{d.get('height')}), fx_fy_cx_cy={d.get('fx_fy_cx_cy')}")
        if "extrinsic_4x4" in d:
            print("extrinsic_4x4:\n", d["extrinsic_4x4"])

calib_map = get_camera_calibration_map(frame)
print("num camera_calibrations:", len(calib_map))
print_camera_calibrations(calib_map)


num camera_calibrations: 8

========== CAMERA CALIBRATIONS ==========

[2] size=(972x1079), fx_fy_cx_cy=[1112.1429443359375, 1112.0059814453125, 488.12847900390625, 719.156005859375]
extrinsic_4x4:
 [[ 0.706 -0.708  0.001  1.445]
 [ 0.708  0.706  0.009  0.153]
 [-0.007 -0.005  1.     1.806]
 [ 0.     0.     0.     1.   ]]

[1] size=(972x1079), fx_fy_cx_cy=[1111.67724609375, 1111.5404052734375, 488.12847900390625, 719.156005859375]
extrinsic_4x4:
 [[ 1.     0.002  0.003  1.519]
 [-0.002  1.     0.008  0.026]
 [-0.003 -0.008  1.     1.806]
 [ 0.     0.     0.     1.   ]]

[3] size=(972x1079), fx_fy_cx_cy=[1110.81982421875, 1110.68310546875, 488.12847900390625, 719.156005859375]
extrinsic_4x4:
 [[ 0.707  0.707  0.005  1.482]
 [-0.707  0.707  0.002 -0.116]
 [-0.002 -0.005  1.     1.806]
 [ 0.     0.     0.     1.   ]]

[4] size=(972x1079), fx_fy_cx_cy=[1110.3507080078125, 1110.2139892578125, 488.12847900390625, 719.156005859375]
extrinsic_4x4:
 [[ 0.    -1.    -0.005  1.303]
 [ 1.    -0.  

### Block 6：通用“列出子 message 的 set/defined 字段”工具

In [8]:
def inspect_submessage(parent_msg, field_name, title=None, show_all_defined=True, max_set_fields=80):
    """
    parent_msg: protobuf message
    field_name: str, e.g., "context" / "pose"
    """
    sub = getattr(parent_msg, field_name, None)
    if sub is None:
        print(f"\n❌ {field_name} not found on this message.")
        return None

    t = title or field_name
    pb_set_fields(sub, t, max_fields=max_set_fields)
    if show_all_defined:
        pb_all_fields(sub, f"{t}")
    return sub


### Block 7：frame.context 深挖（frame 专用）

In [9]:
context = inspect_submessage(frame, "context", title="frame.context", show_all_defined=True)

# 如果你只想看 context 里有哪些字段名（定义层面）
print("\ncontext defined fields:", [f.name for f in context.DESCRIPTOR.fields])

# 如果 camera_calibrations 在里面，就打印一下每个 calibration 是否有 intrinsic/extrinsic
if hasattr(context, "camera_calibrations"):
    print("\nnum camera_calibrations:", len(context.camera_calibrations))
    # 只打印第一个 calibration 的 set fields & all fields，确认结构
    if len(context.camera_calibrations) > 0:
        calib0 = context.camera_calibrations[0]
        pb_set_fields(calib0, "context.camera_calibrations[0] (set)", max_fields=80)
        pb_all_fields(calib0, "context.camera_calibrations[0] (defined)")



=== frame.context (set fields) ===
- name: 5bef8f2e82697e56d4151d1462088b90-206
- camera_calibrations: repeated len=8

=== frame.context (all defined fields) ===
- name: scalar, type=9
- camera_calibrations: repeated, type=11
- laser_calibrations: repeated, type=11
- stats: scalar, type=11

context defined fields: ['name', 'camera_calibrations', 'laser_calibrations', 'stats']

num camera_calibrations: 8

=== context.camera_calibrations[0] (set) (set fields) ===
- name: 2
- intrinsic: repeated len=9
- extrinsic: message<Transform> subfields=1
- width: 972
- height: 1079
- rolling_shutter_direction: 2

=== context.camera_calibrations[0] (defined) (all defined fields) ===
- name: scalar, type=14
- intrinsic: repeated, type=1
- extrinsic: scalar, type=11
- width: scalar, type=5
- height: scalar, type=5
- rolling_shutter_direction: scalar, type=14


### Block 8：frame.pose 与 frame.map_pose_offset（frame 专用）


In [10]:
# 8.1 检查 pose（是否被 set）
pose = inspect_submessage(frame, "pose", title="frame.pose", show_all_defined=True)

# 8.2 检查 map_pose_offset（与地图坐标系相关）
map_pose_offset = inspect_submessage(frame, "map_pose_offset", title="frame.map_pose_offset", show_all_defined=True)



=== frame.pose (set fields) ===

=== frame.pose (all defined fields) ===
- transform: repeated, type=1

=== frame.map_pose_offset (set fields) ===

=== frame.map_pose_offset (all defined fields) ===
- x: scalar, type=1
- y: scalar, type=1
- z: scalar, type=1


### Block 9：frame.labels（先做“存在性探测”，frame 专用）

In [11]:
def inspect_repeated_field(parent_msg, field_name, max_show=3):
    """打印 repeated 字段长度，并对前 max_show 个元素做 set-fields 概览"""
    if not hasattr(parent_msg, field_name):
        print(f"\n❌ {field_name} not found.")
        return None

    val = getattr(parent_msg, field_name)
    try:
        n = len(val)
    except TypeError:
        print(f"\n⚠️ {field_name} exists but has no len() (unexpected).")
        return val

    print(f"\n=== frame.{field_name} === len={n}")
    if n > 0:
        for i in range(min(max_show, n)):
            pb_set_fields(val[i], f"{field_name}[{i}] (set)", max_fields=60)
            pb_all_fields(val[i], f"{field_name}[{i}] (defined)")
    return val

laser_labels = inspect_repeated_field(frame, "laser_labels", max_show=1)
camera_labels = inspect_repeated_field(frame, "camera_labels", max_show=1)
proj_lidar_labels = inspect_repeated_field(frame, "projected_lidar_labels", max_show=1)



=== frame.laser_labels === len=0

=== frame.camera_labels === len=0

=== frame.projected_lidar_labels === len=0


### Block 10：frame.map_features（先做“结构探测”，frame 专用）

In [12]:
map_features = inspect_repeated_field(frame, "map_features", max_show=1)



=== frame.map_features === len=0


In [13]:
print("Total defined fields:", len(frame.DESCRIPTOR.fields))
for f in frame.DESCRIPTOR.fields:
    print(f.name)


Total defined fields: 11
context
timestamp_micros
pose
images
lasers
laser_labels
projected_lidar_labels
camera_labels
no_label_zones
map_features
map_pose_offset


In [14]:
import tensorflow as tf
from collections import defaultdict

def parse_e2ed(raw_bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def get_context_name(frame):
    # Waymo Frame 常见：frame.context.name
    if hasattr(frame, "context") and hasattr(frame.context, "name") and frame.context.name:
        return frame.context.name
    return None

def get_timestamp_micros(frame):
    # 你这条样本显示是 0，但我们还是统一取一下
    if hasattr(frame, "timestamp_micros"):
        return int(frame.timestamp_micros)
    return None

def scan_tfrecord_for_segments(filename, max_records=None):
    ds = tf.data.TFRecordDataset(filename, compression_type=("GZIP" if filename.endswith(".gz") else ""))
    seg = defaultdict(lambda: {"count": 0, "tmin": None, "tmax": None, "idx_first": None, "idx_last": None})

    for i, r in enumerate(ds):
        if max_records is not None and i >= max_records:
            break
        e2e = parse_e2ed(r.numpy())
        fr = e2e.frame

        ctx = get_context_name(fr) or "UNKNOWN_CONTEXT"
        t = get_timestamp_micros(fr)

        s = seg[ctx]
        s["count"] += 1
        if s["idx_first"] is None:
            s["idx_first"] = i
        s["idx_last"] = i

        if t is not None:
            if s["tmin"] is None or t < s["tmin"]:
                s["tmin"] = t
            if s["tmax"] is None or t > s["tmax"]:
                s["tmax"] = t

    # 打印摘要：每个 context 有多少条记录、时间跨度
    print("=== Segment summary in this TFRecord ===")
    for ctx, s in sorted(seg.items(), key=lambda kv: kv[1]["count"], reverse=True)[:10]:
        if s["tmin"] is not None and s["tmax"] is not None and s["tmax"] > s["tmin"]:
            dur = (s["tmax"] - s["tmin"]) / 1e6
            print(f"- {ctx}: n={s['count']}, duration≈{dur:.2f}s, idx=[{s['idx_first']},{s['idx_last']}]")
        else:
            print(f"- {ctx}: n={s['count']}, duration=UNKNOWN (timestamp missing/zero), idx=[{s['idx_first']},{s['idx_last']}]")

    return seg


In [15]:
seg_info = scan_tfrecord_for_segments(FILENAME, max_records=2000)  # 先扫前2000条试试


=== Segment summary in this TFRecord ===
- 5bef8f2e82697e56d4151d1462088b90-206: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[0,0]
- fc9aa7a016420317dae6d3fc3903f401-032: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[1,1]
- 1f4f28e83c40d27675ed1eb4f4e4dacf-172: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[2,2]
- df066120eff5488fb7d9b5304243db77-192: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[3,3]
- b893f870bc2a934a69a172493177c48c-067: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[4,4]
- 0931d95064ce771d7216263d53be126f-078: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[5,5]
- 60a492fa895296ec6c1fac483bc400ca-210: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[6,6]
- 62e779c32b94ff46ede44346d94a97f5-160: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[7,7]
- adba328ccd36b14cb2d1f2c777859861-131: n=1, duration=UNKNOWN (timestamp missing/zero), idx=[8,8]
- 38561ca659f7ba32383f3f1cad122546-213: n=1, duration=UNKNOWN (timestamp miss

In [16]:
def split_context_name(ctx_name: str):
    """
    期望格式：<hex>-<frame_idx>
    返回：(segment_id, frame_idx[int or None])
    """
    if ctx_name is None:
        return None, None
    if "-" not in ctx_name:
        return ctx_name, None
    base, suf = ctx_name.rsplit("-", 1)
    try:
        return base, int(suf)
    except ValueError:
        return ctx_name, None


In [17]:
import tensorflow as tf
from collections import defaultdict

def parse_e2ed(raw_bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def get_context_name(frame):
    if hasattr(frame, "context") and hasattr(frame.context, "name") and frame.context.name:
        return frame.context.name
    return None

def scan_tfrecord_segments_by_base(filename, max_records=None, fps_assumption=10):
    ds = tf.data.TFRecordDataset(filename, compression_type=("GZIP" if filename.endswith(".gz") else ""))

    seg = defaultdict(lambda: {"count": 0, "idx_min": None, "idx_max": None})
    for i, r in enumerate(ds):
        if max_records is not None and i >= max_records:
            break

        e2e = parse_e2ed(r.numpy())
        ctx = get_context_name(e2e.frame) or "UNKNOWN"
        seg_id, frame_idx = split_context_name(ctx)

        s = seg[seg_id]
        s["count"] += 1
        if frame_idx is not None:
            s["idx_min"] = frame_idx if s["idx_min"] is None else min(s["idx_min"], frame_idx)
            s["idx_max"] = frame_idx if s["idx_max"] is None else max(s["idx_max"], frame_idx)

    print("=== Segment summary (grouped by base segment id) ===")
    items = sorted(seg.items(), key=lambda kv: kv[1]["count"], reverse=True)[:10]
    for seg_id, s in items:
        if s["idx_min"] is not None and s["idx_max"] is not None and s["idx_max"] >= s["idx_min"]:
            approx_dur = (s["idx_max"] - s["idx_min"] + 1) / fps_assumption
            print(f"- {seg_id}: n={s['count']} frames, frame_idx=[{s['idx_min']},{s['idx_max']}], duration≈{approx_dur:.2f}s @ {fps_assumption}Hz")
        else:
            print(f"- {seg_id}: n={s['count']} frames, frame_idx=UNKNOWN")
    return seg


In [18]:
seg = scan_tfrecord_segments_by_base(FILENAME, max_records=5000, fps_assumption=10)


=== Segment summary (grouped by base segment id) ===
- 62502fb60c74f34c9edf8ad3c7461776: n=8 frames, frame_idx=[46,228], duration≈18.30s @ 10Hz
- 758d558fd6217582d6601940af85c5cb: n=8 frames, frame_idx=[39,197], duration≈15.90s @ 10Hz
- ef06b076c2f0910d2499e33961803074: n=7 frames, frame_idx=[53,202], duration≈15.00s @ 10Hz
- 1999b9e4fa6865bcc0a3d320cd4384ba: n=7 frames, frame_idx=[49,212], duration≈16.40s @ 10Hz
- fb0ed944efebd34d756103188d59da85: n=7 frames, frame_idx=[20,223], duration≈20.40s @ 10Hz
- d2f02321a3360119781084482e33cf9d: n=7 frames, frame_idx=[86,199], duration≈11.40s @ 10Hz
- 4fe50a077365cafbfd8cc0778110391c: n=7 frames, frame_idx=[53,204], duration≈15.20s @ 10Hz
- 27b862196b866184a3c6b28dd1cdfbba: n=6 frames, frame_idx=[17,201], duration≈18.50s @ 10Hz
- f1dd0e328596edd3c4cd977a0d09aef5: n=6 frames, frame_idx=[63,207], duration≈14.50s @ 10Hz
- fa7cf34b3f20b157a601f3d5b72cab7c: n=6 frames, frame_idx=[81,210], duration≈13.00s @ 10Hz


In [19]:
import cv2
import numpy as np

def cam_name_from_id(cam_id):
    # 你之前已经用 DESCRIPTOR 方式搞定 enum，这里给一个独立版本
    try:
        return open_dataset.CameraName.DESCRIPTOR.values_by_number[int(cam_id)].name
    except Exception:
        return str(cam_id)

def get_camera_image(frame, cam_name="FRONT"):
    # cam_name: "FRONT", "FRONT_LEFT", ...
    cam_id = open_dataset.CameraName.Value(cam_name)
    for img in frame.images:
        if int(img.name) == int(cam_id):
            return tf.io.decode_jpeg(img.image).numpy()  # RGB
    return None

def export_segment_video_from_file(filename, out_mp4, target_seg_id, cam_name="FRONT", fps=10, max_frames=None):
    ds = tf.data.TFRecordDataset(filename, compression_type=("GZIP" if filename.endswith(".gz") else ""))

    frames = []
    for r in ds:
        e2e = parse_e2ed(r.numpy())
        ctx = get_context_name(e2e.frame)
        seg_id, frame_idx = split_context_name(ctx or "")
        if seg_id != target_seg_id:
            continue

        img = get_camera_image(e2e.frame, cam_name=cam_name)
        if img is None:
            continue
        frames.append((frame_idx if frame_idx is not None else 0, img))

        if max_frames is not None and len(frames) >= max_frames:
            break

    if not frames:
        raise RuntimeError("没在这个文件里找到该 segment_id 的帧；可能该 segment 分散在其他 shard。")

    frames.sort(key=lambda x: x[0])  # 按 frame_idx 排序
    h, w = frames[0][1].shape[:2]
    vw = cv2.VideoWriter(out_mp4, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    for _, rgb in frames:
        bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        vw.write(bgr)

    vw.release()
    print(f"✅ Saved: {out_mp4} | frames={len(frames)} | duration≈{len(frames)/fps:.2f}s @ {fps}Hz")


In [26]:
# =========================
# WOD-E2E: Reconstruct a video (single TFRecord shard)
# - Collect frames for a given segment_id from ONE TFRecord file
# - Export a filled video by repeating last frame to fill missing frame_idx
# =========================

import tensorflow as tf
import numpy as np
import cv2

# ---------- Parse ----------
def parse_e2ed(raw_bytes: bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def get_context_name(frame):
    if hasattr(frame, "context") and hasattr(frame.context, "name") and frame.context.name:
        return frame.context.name
    return ""

def split_context_name(ctx_name: str):
    """
    Expected: <segment_hex>-<frame_idx>
    Returns: (segment_id, frame_idx[int or None])
    """
    if not ctx_name or "-" not in ctx_name:
        return ctx_name, None
    base, suf = ctx_name.rsplit("-", 1)
    try:
        return base, int(suf)
    except ValueError:
        return ctx_name, None

# ---------- Camera enum mapping (robust) ----------
def cam_id_from_name(cam_name: str):
    """
    Robust mapping from camera name (e.g., "FRONT") -> enum number,
    without relying on .Value() / .Name() / values_by_name (which can vary by build).
    """
    name = cam_name.split(".")[-1]  # tolerate "CameraName.FRONT"

    # 1) Try protobuf enum wrapper API if exists
    if hasattr(open_dataset.CameraName, "Value"):
        try:
            return int(open_dataset.CameraName.Value(name))
        except Exception:
            pass

    # 2) Try DESCRIPTOR (may be EnumDescriptor, or other)
    desc = getattr(open_dataset.CameraName, "DESCRIPTOR", None)
    if desc is not None:
        # EnumDescriptor usually has .values (list of EnumValueDescriptor)
        values = getattr(desc, "values", None)
        if values is not None:
            for v in values:
                if getattr(v, "name", None) == name:
                    return int(getattr(v, "number"))
        # Some builds expose values_by_number
        vbn = getattr(desc, "values_by_number", None)
        if isinstance(vbn, dict):
            for num, v in vbn.items():
                if getattr(v, "name", None) == name:
                    return int(num)

    # 3) If user passed a number-like string, accept it
    try:
        return int(cam_name)
    except Exception:
        # Print a small hint list if we can
        hint = ""
        if desc is not None and getattr(desc, "values", None) is not None:
            hint = f"\nAvailable names example: {[v.name for v in desc.values[:10]]} ..."
        raise KeyError(f"Unknown camera name: {cam_name}.{hint}")

def get_camera_image_rgb(frame, cam_name="FRONT"):
    """
    Return RGB image np.ndarray(H,W,3) or None
    """
    cam_id = cam_id_from_name(cam_name)
    for img in frame.images:
        if int(img.name) == int(cam_id):
            return tf.io.decode_jpeg(img.image).numpy()  # RGB
    return None

# ---------- Collect frames from ONE file ----------
def collect_segment_frames_from_file(filename, target_seg_id, cam_name="FRONT", max_records=None):
    ds = tf.data.TFRecordDataset(
        filename,
        compression_type=("GZIP" if filename.endswith(".gz") else "")
    )

    frames = []
    for i, r in enumerate(ds):
        if max_records is not None and i >= max_records:
            break

        e2e = parse_e2ed(r.numpy())
        fr = e2e.frame
        ctx = get_context_name(fr)

        seg_id, frame_idx = split_context_name(ctx)
        if seg_id != target_seg_id:
            continue

        rgb = get_camera_image_rgb(fr, cam_name=cam_name)
        if rgb is None:
            continue

        frames.append((frame_idx if frame_idx is not None else -1, rgb))

    # sort + dedup by frame_idx
    frames.sort(key=lambda x: x[0])
    dedup = {}
    for idx, rgb in frames:
        if idx not in dedup:
            dedup[idx] = rgb
    frames = sorted(dedup.items(), key=lambda x: x[0])

    print(f"✅ collected {len(frames)} unique frames from this file")
    if frames:
        print("frame_idx range:", frames[0][0], "~", frames[-1][0])
    return frames

# ---------- Export video (fill gaps by repeating last frame) ----------
def export_video_fill_gaps(frames, out_mp4, fps=10):
    """
    frames: list[(frame_idx, rgb)]
    Fill missing indices by repeating last available frame.
    Duration≈(idx_max-idx_min+1)/fps
    """
    if not frames:
        raise RuntimeError("frames is empty")

    idxs = [i for i, _ in frames]
    if any(i < 0 for i in idxs):
        raise RuntimeError("frame_idx missing (-1). Cannot fill by idx; check context.name format.")

    idx_min, idx_max = idxs[0], idxs[-1]
    table = {i: rgb for i, rgb in frames}

    h, w = frames[0][1].shape[:2]
    vw = cv2.VideoWriter(out_mp4, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    last = table[idx_min]
    written = 0
    for i in range(idx_min, idx_max + 1):
        if i in table:
            last = table[i]
        vw.write(cv2.cvtColor(last, cv2.COLOR_RGB2BGR))
        written += 1

    vw.release()
    print(f"✅ Saved: {out_mp4}")
    print(f"   written={written} frames, duration≈{written/fps:.2f}s @ {fps}Hz")

# =========================
# RUN (edit these 3 lines)
# =========================
SEG_ID = "fb0ed944efebd34d756103188d59da85"  # your ~20.40s example
CAM = "1"                                 # try: "FRONT_LEFT" / "FRONT_RIGHT"
FPS = 10

frames = collect_segment_frames_from_file(FILENAME, SEG_ID, cam_name=CAM)
export_video_fill_gaps(frames, f"{SEG_ID}_{CAM}_filled.mp4", fps=FPS)


✅ collected 7 unique frames from this file
frame_idx range: 20 ~ 223
✅ Saved: fb0ed944efebd34d756103188d59da85_1_filled.mp4
   written=204 frames, duration≈20.40s @ 10Hz


In [27]:
import os, glob, re
import numpy as np
import tensorflow as tf

def parse_e2ed(raw_bytes: bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def split_context_name(ctx_name: str):
    """'<segment>-<idx>' -> (segment_id, idx:int or None)"""
    if not ctx_name or "-" not in ctx_name:
        return ctx_name, None
    base, suf = ctx_name.rsplit("-", 1)
    try:
        return base, int(suf)
    except ValueError:
        return ctx_name, None

def get_keyframe_index(e2e):
    ctx = e2e.frame.context.name if hasattr(e2e.frame, "context") else ""
    seg_id, idx = split_context_name(ctx)
    return seg_id, idx, ctx

def _extract_repeated_scalar_states(states_obj, keys=("pos_x","pos_y","pos_z","vel_x","vel_y","accel_x","accel_y")):
    """
    针对你目前观察到的 EgoTrajectoryStates：很多字段是 repeated scalar（pos_x 等）
    返回 dict[k]=np.array, 以及长度 L（以 pos_x 为准）
    """
    out = {}
    for k in keys:
        if hasattr(states_obj, k):
            v = getattr(states_obj, k)
            # v 是 RepeatedScalarContainer
            out[k] = np.array(list(v), dtype=np.float32)
    L = len(out["pos_x"]) if "pos_x" in out else (len(next(iter(out.values()))) if out else 0)
    return out, L

def extract_record_features(e2e):
    """
    把一条 record 的关键字段抽出来（不做“对齐”，只保留相对窗口）
    """
    seg_id, key_idx, key_id = get_keyframe_index(e2e)

    # intent (int)
    intent = int(getattr(e2e, "intent", -1))

    # past/future (relative window)
    past_dict, past_L = _extract_repeated_scalar_states(getattr(e2e, "past_states", None))
    fut_dict,  fut_L  = _extract_repeated_scalar_states(getattr(e2e, "future_states", None))

    # preference trajectories: 先做“通用摘要提取”
    prefs = []
    if hasattr(e2e, "preference_trajectories"):
        for p in e2e.preference_trajectories:
            d = {}
            if hasattr(p, "preference_score"):
                d["preference_score"] = float(p.preference_score)
            # 如果还有别的字段（比如轨迹点），你后面可以继续加
            prefs.append(d)

    return {
        "seg_id": seg_id,
        "key_idx": key_idx,
        "key_id": key_id,
        "intent": intent,
        "past": past_dict, "past_L": past_L,
        "future": fut_dict, "future_L": fut_L,
        "prefs": prefs,
    }


In [28]:
def list_tfrecords(folder):
    files = sorted(glob.glob(os.path.join(folder, "*.tfrecord*")))
    if not files:
        raise FileNotFoundError(f"No tfrecord under: {folder}")
    return files

def collect_segment_records(val_folder, target_seg_id, max_files=None):
    """
    扫描 val_folder 下所有 tfrecord shard，把属于 target_seg_id 的 record 收集起来
    返回 records: list[dict]（已提取 features）
    """
    files = list_tfrecords(val_folder)
    if max_files is not None:
        files = files[:max_files]

    records = []
    for fp in files:
        ds = tf.data.TFRecordDataset(fp, compression_type=("GZIP" if fp.endswith(".gz") else ""))
        for r in ds:
            e2e = parse_e2ed(r.numpy())
            seg_id, key_idx, _ = get_keyframe_index(e2e)
            if seg_id != target_seg_id:
                continue
            records.append(extract_record_features(e2e))

    # 按 key_idx 排序
    records = [x for x in records if x["key_idx"] is not None]
    records.sort(key=lambda x: x["key_idx"])
    return records


In [29]:
def summarize_segment(records, fps=10):
    if not records:
        print("No records for this segment.")
        return

    idxs = np.array([r["key_idx"] for r in records], dtype=int)
    idx_min, idx_max = int(idxs.min()), int(idxs.max())
    approx_duration = (idx_max - idx_min + 1) / float(fps)

    past_Ls = [r["past_L"] for r in records]
    fut_Ls  = [r["future_L"] for r in records]
    intents = [r["intent"] for r in records]

    print("=== Segment summary ===")
    print("segment_id:", records[0]["seg_id"])
    print(f"keyframes: {len(records)} | key_idx range: [{idx_min}, {idx_max}] | duration≈{approx_duration:.2f}s @ {fps}Hz")
    print("past_L (min/median/max):", int(np.min(past_Ls)), int(np.median(past_Ls)), int(np.max(past_Ls)))
    print("future_L (min/median/max):", int(np.min(fut_Ls)), int(np.median(fut_Ls)), int(np.max(fut_Ls)))

    # intent timeline（粗看是否基本恒定）
    uniq, cnt = np.unique(intents, return_counts=True)
    print("intent counts:", dict(zip([int(u) for u in uniq], [int(c) for c in cnt])))
    # 如果想看变化点：
    changes = []
    for i in range(1, len(records)):
        if records[i]["intent"] != records[i-1]["intent"]:
            changes.append((records[i-1]["key_idx"], records[i]["key_idx"], records[i-1]["intent"], records[i]["intent"]))
    print("intent changes (prev_idx->next_idx, prev_intent->next_intent):", changes[:10])

    # preference_scores 汇总
    scores = []
    for r in records:
        for p in r["prefs"]:
            if "preference_score" in p:
                scores.append(p["preference_score"])
    if scores:
        scores = np.array(scores, dtype=float)
        print("preference_score stats:", {"min": float(scores.min()), "mean": float(scores.mean()), "max": float(scores.max()), "n": int(scores.size)})
    else:
        print("preference_score: none found in these records")


In [30]:
def build_ego_xy_timeline(records, fps=10):
    """
    返回 dict:
      timeline["idx"] = sorted absolute indices
      timeline["x"], timeline["y"] = merged positions (NaN if missing)
    合并策略：同一个 abs_idx 如果出现多次，取平均
    """
    if not records:
        return None

    # 收集 abs_idx -> list[x], list[y]
    agg = {}
    for r in records:
        key = r["key_idx"]
        past = r["past"]
        fut  = r["future"]

        # 约定：past 的最后一个点最接近 key_idx（你可以按需要调整）
        # 这里用 offsets = [-past_L, ..., -1]
        if "pos_x" in past and "pos_y" in past:
            L = len(past["pos_x"])
            offsets = np.arange(-L, 0, dtype=int)
            for off, x, y in zip(offsets, past["pos_x"], past["pos_y"]):
                abs_i = key + int(off)
                agg.setdefault(abs_i, {"x": [], "y": []})
                agg[abs_i]["x"].append(float(x))
                agg[abs_i]["y"].append(float(y))

        # future: offsets = [1,2,...,L]
        if "pos_x" in fut and "pos_y" in fut:
            L = len(fut["pos_x"])
            offsets = np.arange(1, L+1, dtype=int)
            for off, x, y in zip(offsets, fut["pos_x"], fut["pos_y"]):
                abs_i = key + int(off)
                agg.setdefault(abs_i, {"x": [], "y": []})
                agg[abs_i]["x"].append(float(x))
                agg[abs_i]["y"].append(float(y))

    idxs = sorted(agg.keys())
    x = np.array([np.mean(agg[i]["x"]) if agg[i]["x"] else np.nan for i in idxs], dtype=np.float32)
    y = np.array([np.mean(agg[i]["y"]) if agg[i]["y"] else np.nan for i in idxs], dtype=np.float32)

    return {"idx": np.array(idxs, dtype=int), "x": x, "y": y, "fps": fps}


In [31]:
import matplotlib.pyplot as plt

def plot_ego_xy(tl):
    idx = tl["idx"]
    x, y = tl["x"], tl["y"]
    ok = np.isfinite(x) & np.isfinite(y)
    plt.figure(figsize=(6,6))
    plt.plot(x[ok], y[ok])
    plt.title(f"Ego XY (points={ok.sum()}, idx[{idx.min()}..{idx.max()}])")
    plt.axis("equal")
    plt.show()


In [32]:
VAL_DIR = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/val"  # 你自己环境的路径
SEG_ID  = "fb0ed944efebd34d756103188d59da85"

records = collect_segment_records(VAL_DIR, SEG_ID)
summarize_segment(records, fps=10)

tl = build_ego_xy_timeline(records, fps=10)
if tl:
    plot_ego_xy(tl)


=== Segment summary ===
segment_id: fb0ed944efebd34d756103188d59da85
keyframes: 229 | key_idx range: [8, 236] | duration≈22.90s @ 10Hz
past_L (min/median/max): 16 16 16
future_L (min/median/max): 20 20 20
intent counts: {1: 229}
intent changes (prev_idx->next_idx, prev_intent->next_intent): []
preference_score stats: {'min': -1.0, 'mean': -0.9592430858806404, 'max': 10.0, 'n': 687}
